## 7.32 异常和错误处理

运行时错误会导致异常，这些异常以陷阱或软件中断的形式被检测到。
代码中使用`try-catch`块可以捕获这些异常。
如果启用了异常处理，代码却没有`try-catch`块，程序将崩溃并显示错误消息。

异常处理旨在检测很少发生的错误，并以优雅的方式从错误情形中恢复。
您可能认为只要不发生错误，异常处理就不会占用额外的时间，但不幸的是，事实并非总是如此。
程序可能需要进行大量的信息簿记工作，才能知道如何在发生异常时进行恢复。
这种簿记花费的成本在很大程度上取决于不同的编译器。
一些编译器使用高效的基于表的方法，几乎没有额外开销，而其它编译器具有低效的基于代码的方法，或者依赖运行时类型识别（RTTI），会影响代码的其他部分。

进一步解释参照[ISO/IEC TR18015 Technical Report on C++ Performance](http://www.open-std.org/jtc1/sc22/wg21/docs/TR18015.pdf)。

下示例说明了为何需要簿记：

```cpp
// Example 7.48
class C1 {
public:
   ...
   ~C1();
};

void F1() {
   C1 x;
   ...
}

void F0() {
   try {
     F1();
   }
   catch (...) {
     ...
   }
}
```

函数`F1`本该在返回时调用对象`x`的析构函数。
但是如果在`F1`的某个地方发生异常怎么办？ 然后执行跳出了`F1`，函数没有返回。
由于`F1`被中断，因此无法清理`F1`。
现在，异常处理程序负责调用`x`的析构函数。
要可以执行此操作，`F1`必需要保存所有相关的信息：
- 被调用的析构函数的所有信息
- 或可能需要的任何其他清理函数的信息

如果F1调用另一个函数，该函数又调用另一个函数等，并且如果在最里面的函数中发生异常，则异常处理程序需要有关函数调用链的所有信息，并且它需要通过函数调用向后跟踪轨道 检查所有必要的清理工作。
这称为堆栈展开。

如果：
- `F1`调用另一个函数，该函数又调用另一个函数，等等，
- 并且如果在最里面的函数中发生异常

异常处理程序需要有关函数调用链的所有信息，并且它需要沿着函数调用的反向跟踪轨迹，检查所有必要的清理工作。
这称为堆栈展开。

所有函数都必须为异常处理程序保存一些信息，即使没有异常发生。
这就是为什么在某些编译器中异常处理可能代价高昂的原因。
如果你的应用程序异常处理不是必需的，你应该禁用它，这样代码更小，也更高效。
- 你可以通过关闭编译器中的异常处理选项，来禁用整个程序的异常处理。
- 你可以通过向函数原型添加`throw()`来禁用单个函数的异常处理：
```cpp
void F1() throw();
```

This allows the compiler to assume that F1 will never throw any exception so that it doesn't
have to save recovery information for function F1. However, if F1 calls another function F2
that can possibly throw an exception then F1 has to check for exceptions thrown by F2 and
call the std::unexpected() function in case F2 actually throws an exception. Therefore,
you should apply the empty throw() specification to F1 only if all functions called by F1
also have an empty throw() specification. The empty throw()specification is useful for
library functions.

The compiler makes a distinction between leaf functions and frame functions. A frame
function is a function that calls at least one other function. A leaf function is a function that
doesn't call any other function. A leaf function is simpler than a frame function because the
stack unwinding information can be left out if exceptions can be ruled out or if there is
nothing to clean up in case of an exception. A frame function can be turned into a leaf
function by inlining all the functions that it calls. The best performance is obtained if the
critical innermost loop of a program contains no calls to frame functions.

While an empty throw() statement can improve optimizations in some cases, there is no
reason to add statements like throw(A,B,C) to tell explicitly what kind of exceptions a
function can throw. In fact, the compiler may actually add extra code to check that thrown
exceptions are indeed of the specified types (See Sutter: A Pragmatic Look at Exception
Specifications, [http://drdobbs.com/architecture-and-design/184401544, Dr Dobbs Journal, 2002]).

In some cases, it is optimal to use exception handling even in the most critical part of a
program. This is the case if alternative implementations are less efficient and you want to be
able to recover from errors. The following example illustrates such a case:

```cpp
// Example 7.49
// Portability note: This example is specific to Microsoft compilers.
// It will look different in other compilers.
#include <excpt.h>
#include <float.h>
#include <math.h>
#define EXCEPTION_FLT_OVERFLOW 0xC0000091L

void MathLoop() {
 const int arraysize = 1000; unsigned int dummy;
double a[arraysize], b[arraysize], c[arraysize];
 // Enable exception for floating point overflow:
_controlfp_s(&dummy, 0, _EM_OVERFLOW);
 // _controlfp(0, _EM_OVERFLOW); // if above line doesn't work
int i = 0; // Initialize loop counter outside both loops
 // The purpose of the while loop is to resume after exceptions:
 while (i < arraysize) {
 // Catch exceptions in this block:
 __try {
 // Main loop for calculations:
 for ( ; i < arraysize; i++) {
 // Overflow may occur in multiplication here:
 a[i] = log (b[i] * c[i]);
 }
 }
 // Catch floating point overflow but no other exceptions:
 __except (GetExceptionCode() == EXCEPTION_FLT_OVERFLOW
 ? EXCEPTION_EXECUTE_HANDLER : EXCEPTION_CONTINUE_SEARCH) {
 // Floating point overflow has occurred.
 // Reset floating point status:
 _fpreset();
 _controlfp_s(&dummy, 0, _EM_OVERFLOW);
 // _controlfp(0, _EM_OVERFLOW); // if above doesn't work
 // Re-do the calculation in a way that avoids overflow:
 a[i] = log(b[i]) + log(c[i]);
 // Increment loop counter and go back into the for-loop:
 i++;
 }
 }
}
```

Assume that the numbers in b[i] and c[i] are so big that overflow can occur in the
multiplication b[i]*c[i], though this only happens rarely. The above code will catch an
exception in case of overflow and redo the calculation in a way that takes more time but
avoids the overflow. Taking the logarithm of each factor rather than the product makes sure
that no overflow can occur, but the calculation time is doubled.

The time it takes to make support for the exception handling is negligible because there is
no try block or function call (other than log) inside the critical innermost loop. log is a
library function which we assume is optimized. We cannot change its possible exception
handling support anyway. The exception is costly when it occurs, but this is not a problem
since we are assuming that the occurrence is rare.

Testing for the overflow condition inside the loop does not cost anything here because we
are relying on the microprocessor hardware for raising an exception in case of overflow. The
exception is caught by the operating system which redirects it to the exception handler in
the program if there is a try block.

There is a portability issue to catching hardware exceptions. The mechanism relies on nonstandardized details in both compiler, operating system and CPU hardware. Porting such an
application to a different platform is likely to require modifications in the code.

Let's look at the possible alternatives to exception handling in this example. We might check
for overflow by checking if b[i] and c[i] are too big before multiplying them. This would 
require two floating point comparisons, which are relatively costly because they must be
inside the innermost loop. Another possibility is to always use the safe formula a[i] =
log(b[i]) + log(c[i]);. This would double the number of calls to log, and
logarithms take a long time to calculate. If there is a way to check for overflow outside the
loop without checking all the array elements then this might be a better solution. It might be
possible to do such a check before the loop if all the factors are generated from the same
few parameters. Or it might be possible to do the check after the loop if the results are
combined by some formula into a single result.


### Exceptions and vector code

Vector instructions are useful for doing multiple calculations in parallel. This is described in
chapter 12 below. Exception handling does not work well with vector code because a single
element in a vector may cause an exception while perhaps the other vector elements do
not. You may even generate an exception in a not-taken branch because of the way
branches are implemented in vector code. If the code can benefit from vector instructions
then it is better to disable the trapping of exceptions and rely on the propagation of NAN and
INF instead. See chapter 7.34 below. This is further discussed in the document
www.agner.org/optimize/nan_propagation.pdf.

### Avoiding the cost of exception handling

Exception handling is not necessary when no attempt is made to recover from errors. If you
just want the program to issue an error message and stop the program in case of an error
then there is no reason to use try, catch, and throw. It is more efficient to define your
own error-handling function that simply prints an appropriate error message and then calls
exit.

Calling exit may not be safe if there are allocated resources that need to be cleaned up,
as explained below. There are other possible ways of handling errors without using
exceptions. The function that detects an error can return with an error code which the calling
function can use for recovering or for issuing an error message.

It is recommended to use a systematic and well thought-through approach to error handling.
You have to distinguish between recoverable and non-recoverable errors; make sure
allocated resources are cleaned up in case of an error; and make appropriate error
messages to the user.

### Making exception-safe code

Assume that a function opens a file in exclusive mode, and an error condition terminates the
program before the file is closed. The file will remain locked after the program is terminated
and the user will be unable to access the file until the computer is rebooted. To prevent this
kind of problems you must make your program exception safe. In other words, the program
must clean up everything in case of an exception or other error condition. Things that may
need to be cleaned up include:

Memory allocated with new or malloc.

- Handles to windows, graphic brushes, etc.
- Locked mutexes.
- Open database connections.
- Open files and network connections.
- Temporary files that need to be deleted.
- User work that needs to be saved.
- Any other allocated resource.

The C++ way of handling cleanup jobs is to make a destructor. A function that reads or
writes a file can be wrapped into a class with a destructor that makes sure the file is closed.
The same method can be used for any other resource, such as dynamically allocated
memory, windows, mutexes, database connections, etc.

The C++ exception handling system makes sure that all destructors for local objects are
called. The program is exception safe if there are wrapper classes with destructors to take
care of all cleanup of allocated resources. The system is likely to fail if the destructor causes
another exception.

If you make your own error handling system instead of using exception handling then you
cannot be sure that all destructors are called and resources cleaned up. If an error handler
calls exit(), abort(), _endthread(), etc. then there is no guarantee that all
destructors are called. The safe way to handle an unrecoverable error without using
exceptions is to return from the function. The function may return an error code if possible,
or the error code may be stored in a global object. The calling function must then check for
the error code. If the latter function also has something to clean up then it must return to its
own caller, and so on.
